In [41]:
# Import necessary libraries
import random
import pandas as pd
from collections import deque
import matplotlib
matplotlib.use('TkAgg')  # Use the TkAgg backend
import matplotlib.pyplot as plt

# Define constants
NUM_OPERATING_ROOMS = 4  # Number of operating rooms
URGENCY_PRIORITIES = {'Emergency': 3, 'Urgent': 2, 'Elective': 1}  # Urgency level priorities

# Define a class to represent a surgical procedure
class SurgicalProcedure:
    def __init__(self, procedure_id, urgency_level, procedure_name, equipment, surgeon, arrival_time, duration):
        self.procedure_id = procedure_id
        self.urgency_level = urgency_level
        self.priority = URGENCY_PRIORITIES[urgency_level]  # Calculate priority based on urgency level
        self.procedure_name = procedure_name
        self.equipment = equipment
        self.surgeon = surgeon
        self.arrival_time = arrival_time
        self.duration = duration
        self.start_time = None
        self.end_time = None
        self.resources_allocated = False

# Define a sample dataset
sample_dataset = [
    [0, 'Emergency', 'Appendectomy', 'Laparoscope', 'Dr. Smith', 9, 120],
    [1, 'Urgent', 'Cholecystectomy', 'Laparoscope', 'Dr. Johnson', 10, 90],
    [2, 'Elective', 'Prostatectomy', 'Microscope', 'Dr. Williams', 16, 180],
    [3, 'Urgent', 'Hysterectomy', 'Endoscope', 'Dr. Brown', 13, 150],
    [4, 'Emergency', 'Hysterectomy', 'Endoscope', 'Dr. Davis', 3, 150],
    [5, 'Urgent', 'Hysterectomy', 'Endoscope', 'Dr. Miller', 14, 150],
    [6, 'Elective', 'Laparoscopy', 'Laparoscope', 'Dr. Wilson', 6, 60],
    [7, 'Elective', 'Laparoscopy', 'Laparoscope', 'Dr. Anderson', 11, 60],
    [8, 'Emergency', 'Hysterectomy', 'Endoscope', 'Dr. Thompson', 5, 150],
    [9, 'Urgent', 'Cholecystectomy', 'Laparoscope', 'Dr. Moore', 18, 90],
    [10, 'Emergency', 'Cholecystectomy', 'Laparoscope', 'Dr. Taylor', 1, 90],
    [11, 'Elective', 'Laparoscopy', 'Laparoscope', 'Dr. Jackson', 17, 60],
    [12, 'Elective', 'Laparoscopy', 'Laparoscope', 'Dr. White', 15, 60],
    [13, 'Urgent', 'Laparoscopy', 'Laparoscope', 'Dr. Harris', 20, 60],
    [14, 'Urgent', 'Hysterectomy', 'Endoscope', 'Dr. Martin', 8, 150],
    [15, 'Emergency', 'Cholecystectomy', 'Laparoscope', 'Dr. Thompson', 5, 90],
    [16, 'Emergency', 'Prostatectomy', 'Microscope', 'Dr. Garcia', 4, 180],
    [17, 'Emergency', 'Hysterectomy', 'Endoscope', 'Dr. Martinez', 3, 150],
    [18, 'Urgent', 'Cholecystectomy', 'Laparoscope', 'Dr. Robinson', 12, 90],
    [19, 'Urgent', 'Hysterectomy', 'Endoscope', 'Dr. Clark', 13, 150]
]

# Function to load the dataset from the sample data
def load_dataset():
    procedures = []
    for data in sample_dataset:
        procedure_id, urgency_level, procedure_name, equipment, surgeon, arrival_time, duration = data
        procedure = SurgicalProcedure(procedure_id, urgency_level, procedure_name, equipment, surgeon, arrival_time, duration)
        procedures.append(procedure)
    return procedures

# Function to schedule the surgeries using priority scheduling and banker's algorithm
def schedule_surgeries(procedures):
    operating_rooms = [[] for _ in range(NUM_OPERATING_ROOMS)]  # List of operating rooms
    ready_queue = deque()  # Priority queue for ready procedures
    blocked_queue = deque()  # Queue for blocked procedures
    available_resources = {'Laparoscope': 1, 'Endoscope': 1, 'Microscope': 1, 'Ultrasound': 1}  # Available resources
    resource_allocation = []  # List to store resource allocation over time

    # Sort procedures by arrival time
    procedures.sort(key=lambda x: x.arrival_time)

    # Initialize ready queue with procedures at time 0
    for procedure in procedures:
        if procedure.arrival_time == 0:
            ready_queue.append(procedure)

    current_time = 0
    max_time = max(procedure.arrival_time + procedure.duration for procedure in procedures)
    max_iterations = 1000  # Set a reasonable maximum number of iterations
    iteration_count = 0

    while current_time <= max_time and (ready_queue or blocked_queue or any(operating_rooms)):
        # Update resource allocation for the current time
        resource_allocation.append((current_time, available_resources.copy()))

        # Check if any procedures have finished
        for i, room in enumerate(operating_rooms):
            if room and room[0].end_time == current_time:
                completed_procedure = room.pop(0)
                print(f"Procedure {completed_procedure.procedure_name} (ID: {completed_procedure.procedure_id}) completed at time {current_time}. Surgeon: {completed_procedure.surgeon}, Equipment: {completed_procedure.equipment}")
                release_resources(completed_procedure, available_resources)

        # Schedule procedures from ready queue using banker's algorithm
        while ready_queue:
            procedure = ready_queue.popleft()
            if can_allocate_resources(procedure, available_resources):
                allocate_resources(procedure, available_resources)
                procedure.resources_allocated = True
                room_index = find_available_room(operating_rooms)
                if room_index is not None:
                    operating_rooms[room_index].append(procedure)
                    procedure.start_time = current_time
                    print(f"Procedure {procedure.procedure_name} (ID: {procedure.procedure_id}) started at time {current_time}. Surgeon: {procedure.surgeon}, Equipment: {procedure.equipment}")
                    procedure.end_time = current_time + procedure.duration
                else:
                    blocked_queue.append(procedure)
            elif not procedure.resources_allocated:
                blocked_queue.append(procedure)

        # Add new procedures to ready queue
        for procedure in procedures:
            if procedure.arrival_time == current_time:
                ready_queue.append(procedure)

        # Sort ready queue by priority
        ready_queue = deque(sorted(ready_queue, key=lambda x: x.priority, reverse=True))

        # Move procedures from blocked queue to ready queue if resources are available
        for procedure in list(blocked_queue):
            if can_allocate_resources(procedure, available_resources):
                blocked_queue.remove(procedure)
                ready_queue.append(procedure)

        current_time += 1
        iteration_count += 1
        if iteration_count > max_iterations:
            print("Maximum iterations reached. Terminating simulation.")
            break

    return operating_rooms, resource_allocation

# Function to find an available operating room
def find_available_room(operating_rooms):
    for i, room in enumerate(operating_rooms):
        if not room:
            return i
    return None

# Function to allocate resources for a procedure
def allocate_resources(procedure, available_resources):
    if procedure.equipment:
        available_resources[procedure.equipment] -= 1

# Function to release resources after a procedure
def release_resources(procedure, available_resources):
    if procedure.equipment:
        available_resources[procedure.equipment] += 1

# Function to check if resources can be allocated for a procedure
def can_allocate_resources(procedure, available_resources):
    if procedure.equipment is None:
        return True
    else:
        return available_resources.get(procedure.equipment, 0) > 0

# Function to plot the Gantt chart and resource allocation graph
def plot_charts(operating_rooms, resource_allocation):
    fig, axs = plt.subplots(nrows=NUM_OPERATING_ROOMS+1, ncols=1, figsize=(10, 8), sharex=True)

    for i, room in enumerate(operating_rooms):
        procedures = room
        ax = axs[i]

        # Plot Gantt chart
        for j, procedure in enumerate(procedures):
            duration = procedure.end_time - procedure.start_time
            ax.barh(j, duration, left=procedure.start_time, color='blue', alpha=0.7, edgecolor='black')
            ax.text(procedure.start_time + 5, j, f"{procedure.procedure_name} (ID: {procedure.procedure_id})",
                    verticalalignment='center', fontsize=8)
            ax.text(procedure.start_time + 5, j + 0.2, f"Surgeon: {procedure.surgeon}",
                    verticalalignment='center', fontsize=6)
            ax.text(procedure.start_time + 5, j - 0.2, f"Equipment: {procedure.equipment}",
                    verticalalignment='center', fontsize=6)

        ax.set_ylabel(f'Operating Room {i+1}')
        ax.set_yticks(range(len(procedures)))
        ax.set_yticklabels([f"{procedure.procedure_name} (ID: {procedure.procedure_id})" for procedure in procedures])
        ax.grid(True)

    # Plot resource allocation graph
    ax = axs[-1]
    if resource_allocation:
        times, resources = zip(*resource_allocation)
        for resource_name, resource_values in resources[0].items():
            ax.plot(times, [resource[resource_name] for resource in resources], label=resource_name)
        ax.set_xlabel('Time (minutes)')
        ax.set_ylabel('Available Resources')
        ax.set_title('Resource Allocation Over Time')
        ax.legend()
        ax.grid(True)
    else:
        ax.set_xlabel('Time (minutes)')
        ax.set_ylabel('Available Resources')
        ax.set_title('Resource Allocation Over Time')
        ax.text(0.5, 0.5, 'No resource allocation data available', horizontalalignment='center',
                verticalalignment='center', transform=ax.transAxes)

    plt.tight_layout()
    plt.show()

# Main function
def main():
    procedures = load_dataset()
    print("Initial procedures:")
    for procedure in procedures:
        print(f"{procedure.procedure_name} (ID: {procedure.procedure_id}), Arrival Time: {procedure.arrival_time}")
    print()  # Add an empty line for better readability

    operating_rooms, resource_allocation = schedule_surgeries(procedures)
    plot_charts(operating_rooms, resource_allocation)

# Run the main function
if __name__ == '__main__':
    main()

Initial procedures:
Appendectomy (ID: 0), Arrival Time: 9
Cholecystectomy (ID: 1), Arrival Time: 10
Prostatectomy (ID: 2), Arrival Time: 16
Hysterectomy (ID: 3), Arrival Time: 13
Hysterectomy (ID: 4), Arrival Time: 3
Hysterectomy (ID: 5), Arrival Time: 14
Laparoscopy (ID: 6), Arrival Time: 6
Laparoscopy (ID: 7), Arrival Time: 11
Hysterectomy (ID: 8), Arrival Time: 5
Cholecystectomy (ID: 9), Arrival Time: 18
Cholecystectomy (ID: 10), Arrival Time: 1
Laparoscopy (ID: 11), Arrival Time: 17
Laparoscopy (ID: 12), Arrival Time: 15
Laparoscopy (ID: 13), Arrival Time: 20
Hysterectomy (ID: 14), Arrival Time: 8
Cholecystectomy (ID: 15), Arrival Time: 5
Prostatectomy (ID: 16), Arrival Time: 4
Hysterectomy (ID: 17), Arrival Time: 3
Cholecystectomy (ID: 18), Arrival Time: 12
Hysterectomy (ID: 19), Arrival Time: 13



: 